In [3]:
!pip install tiingo
!pip install --upgrade plotly

import scipy.stats as stats
import yfinance as yf
import numpy as np
import random

from pandas.tseries.offsets import BDay
import pandas_datareader as pdr
#from env import API_KEY
import pandas as pd


# Visualization Imports
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio



import warnings
warnings.filterwarnings("ignore")


In [ ]:
from config import API_KEY

In [4]:
#from plotly.offline import init_notebook_mode
#init_notebook_mode(connected=True)
%matplotlib notebook
import matplotlib.pyplot as plt

In [5]:
# Data Collection
from tiingo import TiingoClient
from datetime import datetime, timedelta
SYMBOL ='MSFT'
config = {
    'api_key': API_KEY,
    'session': True  # Use requests.Session
}
# Create a configuration dictionary


client = TiingoClient(config)

# Calculate the start date as today's date minus 2 years
end_date = datetime.now()
start_date = end_date - timedelta(days=20*365)

# Convert dates to string format
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

df = client.get_dataframe(SYMBOL, frequency='daily',startDate=start_date_str, endDate=end_date_str)

In [6]:
df.head()

,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
date,,,,,,,,,,,,
2004-03-03 00:00:00+00:00,26.37,26.55,26.22,26.35,54998700,16.495783,16.608383,16.401951,16.483272,54998700,0.0,1.0
2004-03-04 00:00:00+00:00,26.37,26.42,26.21,26.37,44023200,16.495783,16.527061,16.395695,16.495783,44023200,0.0,1.0
2004-03-05 00:00:00+00:00,26.35,26.60,26.20,26.23,59849800,16.483272,16.639660,16.389440,16.408206,59849800,0.0,1.0
2004-03-08 00:00:00+00:00,25.83,26.35,25.81,26.31,69367800,16.157986,16.483272,16.145475,16.458250,69367800,0.0,1.0
2004-03-09 00:00:00+00:00,25.72,25.97,25.54,25.77,81111700,16.089175,16.245563,15.976576,16.120453,81111700,0.0,1.0


In [7]:
df.reset_index(inplace=True)
df.rename(columns={'index': 'new_index'}, inplace=True)  # Rename the old index column
df['date'] = df['date'].dt.date  # Extract the date part from the datetime index
df

,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,2004-03-03,26.37,26.55,26.22,26.350,54998700,16.495783,16.608383,16.401951,16.483272,54998700,0.0,1.0
1,2004-03-04,26.37,26.42,26.21,26.370,44023200,16.495783,16.527061,16.395695,16.495783,44023200,0.0,1.0
2,2004-03-05,26.35,26.60,26.20,26.230,59849800,16.483272,16.639660,16.389440,16.408206,59849800,0.0,1.0
3,2004-03-08,25.83,26.35,25.81,26.310,69367800,16.157986,16.483272,16.145475,16.458250,69367800,0.0,1.0
4,2004-03-09,25.72,25.97,25.54,25.770,81111700,16.089175,16.245563,15.976576,16.120453,81111700,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5025,2024-02-20,402.79,404.49,398.01,403.240,24307915,402.790000,404.490000,398.010000,403.240000,24307915,0.0,1.0
5026,2024-02-21,402.18,402.29,397.22,400.170,18631072,402.180000,402.290000,397.220000,400.170000,18631072,0.0,1.0
5027,2024-02-22,411.65,412.83,408.57,410.190,27009869,411.650000,412.830000,408.570000,410.190000,27009869,0.0,1.0
5028,2024-02-23,410.34,415.86,408.97,415.670,16295879,410.340000,415.860000,408.970000,415.670000,16295879,0.0,1.0


In [9]:
# Convert the 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

In [10]:
#  extract month and year from dates
df['Month'] = [i.month for i in df['date']]
df['Year'] = [i.year for i in df['date']]
df['Day'] = [i.day for i in df['date']]

In [11]:

df.describe()

,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor,Month,Year,Day
count,5030.000000,5030.000000,5030.000000,5030.000000,5.030000e+03,5030.000000,5030.000000,5030.000000,5030.000000,5.030000e+03,5030.000000,5030.0,5030.000000,5030.000000,5030.000000
mean,91.490900,92.362293,90.540525,91.458664,4.623624e+07,85.373711,86.188065,84.482930,85.343270,4.623624e+07,0.005537,1.0,6.549304,2013.653280,15.736183
std,98.015157,98.957654,96.966309,97.979298,2.867978e+07,99.000925,99.951904,97.945260,98.964545,2.867978e+07,0.063487,0.0,3.425753,5.773537,8.752750
min,15.150000,15.620000,14.870000,15.200000,7.425503e+06,11.285101,11.635200,11.076532,11.322346,7.425503e+06,0.000000,1.0,1.000000,2004.000000,1.000000
25%,27.350000,27.552500,27.150000,27.340000,2.629407e+07,20.256052,20.451147,20.050978,20.251341,2.629407e+07,0.000000,1.0,4.000000,2009.000000,8.000000
50%,38.120000,38.385000,37.805000,38.075000,3.945610e+07,32.181583,32.444075,31.951918,32.124932,3.945610e+07,0.000000,1.0,7.000000,2014.000000,16.000000
75%,113.607500,114.425000,112.385000,113.607500,5.852028e+07,107.504883,108.235902,106.324060,107.533312,5.852028e+07,0.000000,1.0,10.000000,2019.000000,23.000000
max,420.550000,420.820000,415.090000,420.555000,5.910522e+08,419.781151,420.050658,414.331133,419.786142,5.910522e+08,3.080000,1.0,12.000000,2024.000000,31.000000


In [12]:
# forward fill so that days where the market was closed will have the stock price for the previous day
df = df.fillna(method='ffill')
df.head()

,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor,Month,Year,Day
0,2004-03-03,26.37,26.55,26.22,26.35,54998700,16.495783,16.608383,16.401951,16.483272,54998700,0.0,1.0,3,2004,3
1,2004-03-04,26.37,26.42,26.21,26.37,44023200,16.495783,16.527061,16.395695,16.495783,44023200,0.0,1.0,3,2004,4
2,2004-03-05,26.35,26.60,26.20,26.23,59849800,16.483272,16.639660,16.389440,16.408206,59849800,0.0,1.0,3,2004,5
3,2004-03-08,25.83,26.35,25.81,26.31,69367800,16.157986,16.483272,16.145475,16.458250,69367800,0.0,1.0,3,2004,8
4,2004-03-09,25.72,25.97,25.54,25.77,81111700,16.089175,16.245563,15.976576,16.120453,81111700,0.0,1.0,3,2004,9


In [14]:
#Candlestick charts are a visual tool for market analysis, used to describe price movements of a security,
#derivative, or currency. Each "candlestick" typically represents one day of trading and is composed of a body and wicks.

##Body: The wider section of the candlestick which indicates the opening and closing prices.
#If the body is filled or dark, the security closed lower than it opened.
#If the body is empty or light, it closed higher than it opened.

##Wicks: Lines that extend from the top and bottom of the body representing the high and low prices during the period.

In [20]:
def plot_candlestick(stock_df, name='', rolling_avg=None, fig_size=(1100, 700)):
    """
    Plot a candlestick chart for the given stock dataframe with optional rolling averages.

    Args:
        stock_df (pd.DataFrame): The stock data as a pandas DataFrame.
        name (str): The name of the stock, defaults to 'Tesla'.
        rolling_avg (list of int, optional): A list of integers for rolling average window sizes.
        fig_size (tuple): The figure size, defaults to (1100, 700).
    """
    # Copy df to avoid modifying the original data
    stock_data = stock_df.copy()

    # Creating plot
    fig = go.Figure(data=[go.Candlestick(x=stock_data.index,
        close=stock_data['close'], open=stock_data['open'], high=stock_data['high'], low=stock_data['low'],
        name="Candlesticks", increasing_line_color='green', decreasing_line_color='red', line=dict(width=1)
                                        )])
    # Rolling averages if specified
    if rolling_avg:
        colors = ['rgba(0, 255, 255, 0.5)',   # cyan
                  'rgba(255, 255, 0, 0.5)',   # yellow
                  'rgba(255, 165, 0, 0.5)',   # orange
                  'rgba(255, 105, 180, 0.5)', # pink
                  'rgba(165, 42, 42, 0.5)',   # brown
                  'rgba(128, 128, 128, 0.5)', # gray
                  'rgba(128, 128, 0, 0.5)',   # olive
                  'rgba(0, 0, 255, 0.5)']     # blue

        for i, avg in enumerate(rolling_avg):
            color = colors[i % len(colors)]
            ma_column = f'{avg}-day MA'
            stock_data[ma_column] = stock_data['close'].rolling(window=avg).mean()

            # Moving average trace
            fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data[ma_column],
                    mode='lines', name=f'{avg}-day Moving Average', line=dict(color=color)))

    # Layout updates
    fig.update_layout(
    #title=f"{name} Stock Price - Candlestick Chart",
    xaxis_title="Date", yaxis_title="Price",
    width=fig_size[0], height=fig_size[1],
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=14, label="2w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(count=2, label="2y", step="year", stepmode="backward"),
                dict(count=3, label="3y", step="year", stepmode="backward"),
                dict(count=5, label="5y", step="year", stepmode="backward"),
                dict(label="All", step="all")
            ]),
            x=0.1, y=1.1, # Adjust the position of the range selector
            bgcolor='pink',
            font=dict(color='black'),
            activecolor='lightgreen'
        )
    )
)
    fig.show()

In [16]:
df.set_index("date",inplace=True)

In [21]:

# plot out the general Apple stocks into the Candlestick charts

SYMBOL ='MSFT'
plot_candlestick(df, name=SYMBOL)

# With Moving averages
plot_candlestick(df, name=SYMBOL, rolling_avg=[20, 50, 200])

In [22]:
import plotly.graph_objects as go

def plot_candlestick2(stock_df, name='', latestdays=10, rolling_avg=None, fig_size=(1100, 700)):
    """
    Plot a candlestick chart for the given stock dataframe with optional rolling averages.

    Args:
        stock_df (pd.DataFrame): The stock data as a pandas DataFrame.
        name (str): The name of the stock, defaults to 'Tesla'.
        rolling_avg (list of int, optional): A list of integers for rolling average window sizes.
        fig_size (tuple): The figure size, defaults to (1100, 700).
    """
    # Copy df to avoid modifying the original data
    stock_data = stock_df.copy()
    stock_data = stock_df[0-latestdays:]

    # Creating plot
    fig = go.Figure(data=[go.Candlestick(x=stock_data.index,
                                         close=stock_data['close'], open=stock_data['open'],
                                         high=stock_data['high'], low=stock_data['low'],
                                         name="Candlesticks", increasing_line_color='green',
                                         decreasing_line_color='red', line=dict(width=1))])

    # Rolling averages if specified
    if rolling_avg:
        colors = ['rgba(0, 255, 255, 0.5)', 'rgba(255, 255, 0, 0.5)', 'rgba(255, 165, 0, 0.5)',
                  'rgba(255, 105, 180, 0.5)', 'rgba(165, 42, 42, 0.5)', 'rgba(128, 128, 128, 0.5)',
                  'rgba(128, 128, 0, 0.5)', 'rgba(0, 0, 255, 0.5)']

        for i, avg in enumerate(rolling_avg):
            color = colors[i % len(colors)]
            ma_column = f'{avg}-day MA'
            stock_data[ma_column] = stock_data['close'].rolling(window=avg).mean()

            # Moving average trace
            fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data[ma_column],
                                     mode='lines', name=f'{avg}-day Moving Average', line=dict(color=color)))

    # Layout updates
    fig.update_layout(
        #title=f"{name} Stock Price - Candlestick Chart",
        xaxis_title="Date", yaxis_title="Price",
        width=fig_size[0], height=fig_size[1],
        xaxis=dict(range=[stock_data.index[0-latestdays], stock_data.index[-1]])  # Display only the last 7 days
    )
    fig.show()

In [23]:
# plot out the general Apple stocks into the Candlestick charts

SYMBOL ='MSFT'
plot_candlestick2(df, name=SYMBOL,latestdays=14 )

# With Moving averages
#plot_candlestick2(df, name=SYMBOL, rolling_avg=[20, 50, 200])